Exercise 4. In the web of Figure 2.1, remove the link from page 3 to page 1. In the resulting
web page 3 is now a dangling node. Set up the corresponding substochastic matrix and find its largest
positive (Perron) eigenvalue. Find a non-negative Perron eigenvector for this eigenvalue, and scale
the vector so that components sum to one. Does the resulting ranking seem reasonable?

In [1]:
import numpy as np

In [ ]:
# 1. Matrice A con page 3 dangling (esercizio 4)
A = np.array([
    [0.0,   0.0, 0.0, 0.5 ],   # row 1
    [1/3,   0.0, 0.0, 0.0 ],   # row 2
    [1/3, 0.5,  0.0, 0.5 ],    # row 3
    [1/3, 0.5,  0.0, 0.0 ]     # row 4
])

# 2. Calcolo autovalori e autovettori
eigvals, eigvecs = np.linalg.eig(A)

# 3. Trova il Perron eigenvalue (massimo autovalore reale)
# (filtriamo per la parte reale, ignorando eventuale rumore complesso numerico)
eigvals_real = eigvals.real
idx_max = np.argmax(eigvals_real)          # indice dell'autovalore massimo
lambda_perron = eigvals_real[idx_max]      # Perron eigenvalue
v_perron = eigvecs[:, idx_max].real        # autovettore corrispondente (parte reale)

# 4. Rendiamo l'autovettore non-negativo (se cambia solo per segno)
if np.sum(v_perron) < 0:
    v_perron = -v_perron

# 5. Normalizziamo per avere un vettore di probabilità (somma = 1)
pagerank = v_perron / np.sum(v_perron)

print("Autovalori:", eigvals)
print("Perron eigenvalue (massimo):", lambda_perron)
print("Autovettore Perron (non normalizzato):", v_perron)
print("PageRank normalizzato (somma = 1):", pagerank)
print("Somma dei componenti:", np.sum(pagerank))


Exercise 11. Consider again the web in Figure 2.1, with the addition of a page 5 that links to
page 3, where page 3 also links to page 5. Calculate the new ranking by finding the eigenvector of
M (corresponding to λ = 1) that has positive components summing to one. Use m = 0.15.

In [ ]:

# -----------------------------
# 1. Matrice A (web Figura 2.1 + pagina 5 ↔ 3)
# -----------------------------
# Link:
# 1 -> 2,3,4   (n1 = 3)
# 2 -> 3,4     (n2 = 2)
# 3 -> 1,5     (n3 = 2)
# 4 -> 1,3     (n4 = 2)
# 5 -> 3       (n5 = 1)

A = np.array([
    [0.0,   0.0, 0.5, 0.5, 0.0],   # row 1 (page 1)
    [1/3,   0.0, 0.0, 0.0, 0.0],   # row 2 (page 2)
    [1/3, 0.5,  0.0, 0.5, 1.0],    # row 3 (page 3)
    [1/3, 0.5,  0.0, 0.0, 0.0],    # row 4 (page 4)
    [0.0,   0.0, 0.5, 0.0, 0.0],   # row 5 (page 5)
], dtype=float)

n = A.shape[0]

# -----------------------------
# 2. Matrice S (tutta 1/n)
# -----------------------------
S = np.full((n, n), 1.0 / n)

# -----------------------------
# 3. Matrice M = (1-m)A + mS
# -----------------------------
m = 0.15
M = (1 - m) * A + m * S

# (check opzionale: le colonne devono sommare a 1)
print("Somma colonne di M:", M.sum(axis=0))

# -----------------------------
# 4. Power iteration per PageRank
# -----------------------------
def pagerank(M, tol=1e-10, max_iter=1000):
    n = M.shape[0]
    # vettore iniziale uniforme
    x = np.ones(n) / n

    for k in range(max_iter):
        x_new = M @ x
        # differenza in norma 1
        diff = np.linalg.norm(x_new - x, 1)
        # print(f"iter {k}, diff = {diff}")  # se vuoi vedere la convergenza
        if diff < tol:
            return x_new, k + 1
        x = x_new

    return x, max_iter  # se non converge entro max_iter

x_star, iters = pagerank(M)
# normalizzo comunque per sicurezza (anche se dovrebbe già sommare a 1)
x_star = x_star / x_star.sum()

print("\nPageRank trovato in", iters, "iterazioni:")
for i, val in enumerate(x_star, start=1):
    print(f"Pagina {i}: {val:.6f}")

# -----------------------------
# 5. Ordina le pagine per importanza
# -----------------------------
ranking = np.argsort(-x_star)  # ordina decrescente
print("\nRanking (dalla più importante):")
for pos, idx in enumerate(ranking, start=1):
    print(f"{pos}) Pagina {idx+1} con score {x_star[idx]:.6f}")
